# Setting up an OpenMM iMD simulation

This notebook demonstrates how to set up an OpenMM simulation for use with NanoVer from scratch.
We take AMBER files for neuraminidase with oseltamivir (AKA tamiflu) bound, create an OpenMM system and 
set it up with NanoVer, using ASE as the integrator

## Set up the OpenMM Simulation

We start by creating an OpenMM simulation from our AMBER files. OpenMM also supports Gromacs and CHARMM files, and can be customized for many other uses. 

In [1]:
import openmm as mm
import openmm.unit as unit 
import openmm.app as app

In [2]:
prmtop = app.AmberPrmtopFile("openmm_files/3TI6_ose_wt.top")
amber_coords = app.AmberInpcrdFile("openmm_files/3TI6_ose_wt.rst")

Because we use ASE for integrating, we keep the simulation simple by using implicit solvent and no constraints

In [3]:
system = prmtop.createSystem(nonbondedMethod=app.CutoffPeriodic, 
                             nonbondedCutoff=2*unit.nanometer, 
                             implicitSolvent=app.OBC2,
                             constraints=None)

C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\openmm\app\internal\amber_file_parser.py:1168: UserWarning: Non-optimal GB parameters detected for GB model OBC2
  warnings.warn(


In [4]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.001*unit.picoseconds)

Create an OpenMM simulation out of the topology, system and integrator

In [5]:
simulation = app.Simulation(prmtop.topology, system, integrator)

In [6]:
simulation.context.setPositions(amber_coords.positions)
if amber_coords.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*amber_coords.boxVectors)

Minimize the energy to create a stable conformation

In [7]:
simulation.minimizeEnergy()

Run a few steps to check it's stable

In [8]:
simulation.context.setVelocitiesToTemperature(300 * unit.kelvin)

In [9]:
import sys

In [10]:
simulation.reporters.append(app.StateDataReporter(sys.stdout, 100, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,-39543.773704749576,171.94629506417792
200,-37456.68643973493,172.30788716141828
300,-36960.350029212466,185.9325771671761
400,-36279.01056693219,197.09489585675647
500,-35987.88951895856,211.20127489471548
600,-35227.428032142154,219.9709517508048
700,-34626.85153983258,225.99942530207284
800,-34018.469032508365,231.04540015999513
900,-33447.22238944196,238.21879439965178
1000,-33096.13106558942,244.96548097108987


Looks good! Now, let's set it up for use with NanoVer.

The following cell outputs the system as an OpenMM XML file + a PDB file with the topology. This let's you take what you've done here and use it straight in NanoVer elsewhere, perfect if you just want to run a simulation quickly: 

```bash 
nanover-omm-ase neuraminidase_nanover.xml
```

In [11]:
from nanover.openmm.serializer import serialize_simulation

with open('neuraminidase_nanover.xml','w') as f:
    f.write(serialize_simulation(simulation))

In the following section, we'll set up the simulation by hand, essentially replicating what `nanover-omm-ase` does.

## Setting up an OpenMM simulation with ASE and NanoVer

Now that we've got an OpenMM simulation, let's pair it with ASE so we can do interactive molecular dynamics with NanoVer.

**Note**: Here we use OpenMM to calculate the forces and ASE as the integrator. The `narupa-openmm` package lets you use OpenMM for the integrator as well. See the `nanover-omm-server` command.

### Creating the ASE System

We use the OpenMMCalculator to take our simulation and produce an ASE Atoms object

In [12]:
from nanover.ase.openmm import OpenMMCalculator

In [13]:
calculator = OpenMMCalculator(simulation)

In [14]:
atoms = calculator.generate_atoms()
atoms.set_calculator(calculator)
len(atoms)

C:\Users\ragzo\AppData\Local\Temp\ipykernel_10760\2943926735.py:2: DeprecationWarning: Please use atoms.calc = calc
  atoms.set_calculator(calculator)


5880

Now we've got an ASE Atoms object, and a calculator, we can set up NanoVer with ASE as [usual](./ase_basic_example.ipynb). 
The only difference here is that we swap out the default way of sending frames with a specially made one, `openmm_ase_frame_adaptor`, for OpenMM that knows about OpenMM topology

In [15]:
from ase.md import Langevin
import ase.units as ase_units
dynamics = Langevin(atoms, timestep=1.0 * ase_units.fs, temperature_K=300 * ase_units.kB, friction=1.0e-03)

In [17]:
from nanover.ase.openmm.frame_adaptor import openmm_ase_atoms_to_frame_data
from nanover.omni import OmniRunner
from nanover.omni.ase import ASESimulation

In [18]:
omni_sim = ASESimulation.from_ase_dynamics(
    dynamics, 
    ase_atoms_to_frame_data=openmm_ase_atoms_to_frame_data,
)
omni = OmniRunner.with_basic_server(omni_sim)

As always, let's run a few steps to make sure everything looks good

In [19]:
omni_sim.dynamics.run(100)
omni_sim.atoms.get_potential_energy()

-432.65624999030933

All good! Let's leave it running in the background

In [20]:
omni.next()

Connect to it from VR and you'll see something like this:

![NanoVer neuraminidase](./images/neuraminidase_ball_and_stick.png)

## Let's make it pretty!

Ball and stick is so 2001, let's make it look cool. We'll also make it so if you interact with the oseltamivir, it'll be interacted with as a group, which is more stable

First, we connect a client so we can modify the shared state

In [22]:
from nanover.app import NanoverImdClient
client = NanoverImdClient.connect_to_single_server(port=omni.app_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame();

We define a couple of handy methods for playing with selections and colour gradients

In [23]:
import matplotlib.cm

def get_matplotlib_gradient(name: str):
    cmap = matplotlib.colormaps[name]
    return list(list(cmap(x/7)) for x in range(0, 8, 1))

In [24]:
from nanover.mdanalysis import frame_data_to_mdanalysis
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

Hide the 'root' selection, it's getting in the way of our creativity 

In [25]:
root_selection = client.root_selection
with root_selection.modify():
    root_selection.hide = True
    root_selection.interaction_method = 'none'

Let's select the protein

In [26]:
protein = client.create_selection("Protein", [])

In [27]:
with protein.modify():
    protein.set_particles(generate_mdanalysis_selection("protein and not type H"))

We'll colour it and render with a spline, or ribbon, renderer.
Some things you can try: 
* Change the render: `spline`, `geometric spline`. Or comment out the `sequence` line and try `liquorice`,`noodles`, `cycles`, `ball and stick`.
* Change the color: set it to be one color, or try some different matplotlib [color maps](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html), e.g. `rainbow` or `magma`.
* Change the scale.

In [28]:
with protein.modify():
    protein.renderer = {
            'sequence': 'polypeptide',
            'color': {
                'type': 'residue index in entity',
                'gradient': get_matplotlib_gradient('rainbow')
            },
            'render': 'geometric spline',
            'scale': 0.2
        }
    protein.interaction_method = 'single'

Let's reintroduce the ligand, oseltamivir, and make it so we interact with it as a group 

In [29]:
# Select ligand
ligand = client.create_selection("Ligand", [])
with ligand.modify():
    ligand.set_particles(generate_mdanalysis_selection("resname OSE"))

In [30]:
with ligand.modify():
    ligand.renderer = {
            'color': 'cpk',
            'scale': 0.1,
            'render': 'liquorice'
        }
    ligand.velocity_reset = True
    ligand.interaction_method = 'group'

If you've done all that, you'll have something that looks like this:

![Neuraminidase Geometric](./images/neuraminidase_geometric_spline.png)

# Tidying Up After Yourself

In [33]:
client.close()
omni.close()

# Next Steps

* Set up an OpenMM simulation of [graphene](./ase_openmm_graphene.ipynb) with restraints and add UI and custom commands in the notebook 